<a href="https://colab.research.google.com/github/motuzov/ir_embeddings/blob/main/MiniLM_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install 'transformers[torch]'

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

In [ ]:
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [ ]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

In [ ]:
# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)

In [ ]:
! ls -a ~/.cache/huggingface/hub

In [ ]:
save_path = 'all-MiniLM-L6-v2'

In [ ]:
! mkdir {save_path}

In [ ]:
! ls

In [ ]:
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

In [ ]:
! ls all-MiniLM-L6-v2

In [ ]:
model = AutoModel.from_pretrained(save_path, return_dict=True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(save_path)